# Full revision analysis

## List of work to be performed:
1. Supplementary figure showing incorrect inform tumor segmentation and NKT cells 
2. Supplementary figure with zoom-in images of individual cells from inForm and TME-Analyzer in an image with high disagreement 
3. Incorporation of DeepCell into TME-Analyzer 
4. Reorganize the MonteCarlo trials 
5. Supplementary table 6 with full ranking of parameters and discovery/validation p values if top n parameters were used instead. 
6. Generate forest plot for 4E 
7. Build classifier based on train-test split of discovery cohort. Try 16-47, 32-31, 47-16 splits. 
8. ? It is significant amount of work, but since both reviewers mention is, I can train a random forest on the original dataset and report it ? 

## 2. Supplementary figure with zoom-in images of individual cells from inForm and TME-Analyzer in an image with high disagreement 
Stepwise approach;
2.1. Find an image with large disagreement; defined as the geometric mean of the absolute disagreement in number for all phenotypes

In [82]:
import pandas as pd
import numpy as np
import os
import shutil

TME_analyzer_folder = "E:/Desktop_backup_20231031/Desktop/TME-Analyzer_the_manuscript/stardist_analysis_20201108/analysis_results/"
inForm_folder = "E:/TME-Analyzer-large-files-from-desktop-folder/20231207_revision_files/Task1/inForm_output_images/"
output_folder1 = "E:/TME-Analyzer-large-files-from-desktop-folder/20231207_revision_files/Task1/confusion_images/"
output_folder2 = "E:/TME-Analyzer-large-files-from-desktop-folder/20231207_revision_files/Task1/non_analyzed_images/"
px_size = 0.4999

combined_frame5 = pd.read_csv("E:/Moved from E/TME-Analyzer_the_manuscript/20211129_TME-Analyzer/data_analysis/reproduction_of_old_analysis/20230607_Marcel/combined_full.xls",sep='\t').set_index('sample id')

absolute_disagreement = []
for i_index in combined_frame5.index:
    disagreement = 0
    for i in combined_frame5.keys():
        try:
            if ((('inform' in i) | ('inForm' in i)) & ('in All' in i) & ('number' in i)):
                x = combined_frame5.loc[i_index,i]
                y = combined_frame5.loc[i_index,i.replace('inForm','auto')]
                if x + y  > 0:
                    disagreement += np.abs(x-y)/(x+y)
                elif x + y == 0:
                    disagreement += 0
                else:
                    disagreement = np.nan
        except:
            disagreement = np.nan
    absolute_disagreement.append([i_index, disagreement])
absolute_disagreement = np.array(absolute_disagreement)
absolute_disagreement_sorted = absolute_disagreement[absolute_disagreement[:, 1].argsort()]
print(absolute_disagreement_sorted)


[['DZ-88-07849 ign_[44663,18278]' '0.10252911533751566']
 ['RS-90-13354 ign_[55000,12622]' '0.5157568541275825']
 ['DH-96-02660_[56796,9250]' '0.5352980137635901']
 ...
 ['DH-93-07884_[45527,5553]' 'nan']
 ['ZR-89-02083_[40141,12993]' 'nan']
 ['DH-98-01114_[42543,16440]' 'nan']]


Define display composite image

In [94]:
from matplotlib import pyplot as plt
import numpy as np
color_LUT = {"red": [1, 0, 0], "green": [0, 1, 0], "blue": [0, 0, 1], "cyan": [0, 1, 1], "magenta": [1, 0, 1],
             "yellow": [1, 1, 0], "white": [1, 1, 1], "gray": [0.5, 0.5, 0.5], "black": [0, 0, 0], "aquamarine": [0.5, 1, 0.83],
             "coral": [1, 0.5, 0.3], "crimson": [0.85, 0.1, 0.3], "gold": [1, 0.83, 0], "lavender": [0.9, 0.9, 0.98],
             "olive": [0.5, 0.5, 0], "orange": [1, 0.5, 0], "orchid": [0.85, 0.44, 0.84], "pink": [1, 0.75, 0.8], "teal": [0, 0.5, 0.5]}

def display_composite_image(im_raw, ch_colors, Cell_props, pheno_names, pheno_colors):
    plt.figure()
    n_channels = len(ch_colors)
    im_2_display = np.zeros((im_raw.shape[0],
                                im_raw.shape[1], 3), dtype=np.float32)
    for i in range(n_channels):
        Color_pointers_temp = ch_colors[i]
        for j in range(3):
            im_temp = im_2_display[:, :, j]
            im2_add = (im_raw[:, :, i]/im_raw[:, :, i].max())
            im2_add = im2_add * color_LUT[Color_pointers_temp][j]
            im_temp = im_temp + im2_add
            im_2_display[:, :, j] = im_temp
    plt.imshow(im_2_display/2)
    for i in range(len(pheno_names)):
        pheno_name = pheno_names[i]
        if pheno_name == 'CD4':
            Cells_of_interest = [i for i in Cell_props.index if ('CD3+' in Cell_props.loc[i,'Phenotypes']) & ('CD8+' not in Cell_props.loc[i,'Phenotypes'])]
        elif pheno_name == 'CD8':
            Cells_of_interest = [i for i in Cell_props.index if ('CD3+' in Cell_props.loc[i,'Phenotypes']) & ('CD8+' in Cell_props.loc[i,'Phenotypes'])]
        elif pheno_name == 'other':
            Cells_of_interest = [i for i in Cell_props.index if (['All'] == Cell_props.loc[i,'Phenotypes'])]
        else:
            Cells_of_interest = [i for i in Cell_props.index if ((pheno_name + '+') in Cell_props.loc[i,'Phenotypes'])]
        cell_locs = Cell_props.loc[Cells_of_interest,'Cell Centroid'].to_numpy()
        y =[j[0] for j in cell_locs]
        x =[j[1] for j in cell_locs]
        plt.scatter(x,y,s=3,marker='o',color=pheno_colors[i])
    plt.axis('off')
    
            


In [95]:
import pickle

workplace_folder = "E:/Moved from E/TME-Analyzer_the_manuscript/20211129_TME-Analyzer/workplaces/"
workplaces = [i for i in os.listdir(workplace_folder)]
inForm_folder = "E:/TME-Analyzer-large-files-from-desktop-folder/Analysis Effector TNBC/image_with_phenotype_map/"
inform_images = [i for i in os.listdir(inForm_folder)]
output_folder = "E:/TME-Analyzer-large-files-from-desktop-folder/20231207_revision_files/Task2/sorted_phenotype_images/"
# pickle_data = pickle.load(open(workplace_folder + workplaces[0], "rb"))
for n_i, arg_index in enumerate(absolute_disagreement_sorted):
    arg_index = arg_index[0]
    i = [i for i in workplaces if arg_index in i][0]
    pickle_data = pickle.load(open(workplace_folder + i, "rb"))
    display_composite_image(pickle_data['im_raw'],pickle_data['Color_pointers'][:7],
                            pickle_data['Cell_props'],['CK','CD20','CD8','CD4','CD68','CD56','other'],
                            ['cyan','yellow','red','lime','orange','magenta','blue'])
    extent = plt.gca().get_window_extent().transformed(plt.gcf().dpi_scale_trans.inverted())
    plt.savefig(output_folder + f"{n_i:04d}_" + i + '.png', bbox_inches=extent)
    plt.close()
    i = [i for i in inform_images if arg_index in i][0]
    shutil.copy(inForm_folder + i,output_folder + f"{n_i:04d}_" + i)
    

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

IndexError: list index out of range

In [99]:
absolute_disagreement_sorted[n_i]


array(['ZR-89-02083_[44615,8611]', 'nan'], dtype='<U33')